In [7]:
import numpy as np
import pandas as pd
import datetime
from iexfinance.stocks import get_historical_data
from iexfinance.refdata import get_symbols
import matplotlib.pyplot as plt
%matplotlib inline

In [38]:
ticker = "AAPL"


end_date = datetime.date(2018,12,31)
start_date = datetime.date(2017, 1, 1)

# Get 1 year's worth of historical data for AAPL
df = get_historical_data(ticker, start_date, end_date,  output_format='pandas')
df.head()

,open,high,low,close,volume
date,,,,,
2017-01-03,115.80,116.33,114.76,116.15,28781865
2017-01-04,115.85,116.51,115.75,116.02,21118116
2017-01-05,115.92,116.86,115.81,116.61,22193587
2017-01-06,116.78,118.16,116.47,117.91,31751900
2017-01-09,117.95,119.43,117.94,118.99,33561948


In [15]:
import googlefinance
import yahoofinancials

In [18]:
from yahoofinancials import YahooFinancials

ticker = 'AAPL'
yahoo_financials = YahooFinancials(ticker)

balance_sheet_data_qt = yahoo_financials.get_financial_stmts('quarterly', 'balance')
income_statement_data_qt = yahoo_financials.get_financial_stmts('quarterly', 'income')
all_statement_data_qt =  yahoo_financials.get_financial_stmts('quarterly', ['income', 'cash', 'balance'])
apple_earnings_data = yahoo_financials.get_stock_earnings_data()
apple_net_income = yahoo_financials.get_net_income()
historical_stock_prices = yahoo_financials.get_historical_price_data('2008-09-15', '2018-09-15', 'daily')


In [41]:
#
# THIS IS WHAT WE CAN USE!!!!
#
#

from pandas_datareader import data

def
df = data.DataReader('SPY', 'yahoo', '2005-01-01', '2018-12-31').filter(['Date','Adj Close'])
df.rename(
    columns={'Adj Close':'SPY'}, 
    inplace=True
)
print(df)

                   SPY
Date                  
2005-01-03   89.905312
2005-01-04   88.806694
2005-01-05   88.193878
2005-01-06   88.642288
2005-01-07   88.515228
...                ...
2018-12-24  232.183060
2018-12-26  243.914062
2018-12-27  245.786697
2018-12-28  245.469635
2018-12-31  247.619644

[3523 rows x 1 columns]


In [78]:
def fetch_prices_as_df(ticker,start,end):
    """ Fetch ticker adjusted closing prices for start-end date range"""
    data.DataReader(ticker,'yahoo',start,end).filter(['Date','Adj Close'])
    df.rename(columns={'Adj Close': ticker},inplace=True)
    return df

In [79]:
spy_df = fetch_prices_as_df('SPY','2005-01-01', '2018-12-31')
spy_df

,SPY
Date,
2005-01-03,89.905312
2005-01-04,88.806694
2005-01-05,88.193878
2005-01-06,88.642288
2005-01-07,88.515228
...,...
2018-12-24,232.183060
2018-12-26,243.914062
2018-12-27,245.786697


In [83]:
def combine_dfs(original_df,new_df):
    return pd.concat([original_df,new_df],axis="columns", join="inner")

In [88]:
tickers = ['SPY','AGG']
start = '2005-01-01'
end = '2018-12-31'

big_daddy = None

for ticker in tickers:
    big_daddy = combine_dfs(big_daddy,fetch_prices_as_df(ticker,start,end))

print(big_daddy)

                   SPY         SPY
Date                              
2005-01-03   89.905312   89.905312
2005-01-04   88.806694   88.806694
2005-01-05   88.193878   88.193878
2005-01-06   88.642288   88.642288
2005-01-07   88.515228   88.515228
...                ...         ...
2018-12-24  232.183060  232.183060
2018-12-26  243.914062  243.914062
2018-12-27  245.786697  245.786697
2018-12-28  245.469635  245.469635
2018-12-31  247.619644  247.619644

[3523 rows x 2 columns]
